### Inference Model Denoising & Classification


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import models
from AdaptiveFilterWrapper import adapfilt_modeler

In [ ]:
denoising  = 'nlms' # change to 'lms' / 'nlms' / 'rls'

In [ ]:
def read_adapfilt_model(filename, path=""):
    with open(os.path.join(path, filename), 'rb') as in_name:
        model = pickle.load(in_name)
        return model

- load model

In [ ]:
lms_model = read_adapfilt_model("LMS_adapfilt_model_afdb.pkl")
nlms_model = read_adapfilt_model("NLMS_adapfilt_model_afdb.pkl")
rls_model = read_adapfilt_model("RLS_adapfilt_model_afdb.pkl")

In [ ]:
cnn_lms_model = models.load_model('CNN_Classification_model_lms.h5')
cnn_nlms_model = models.load_model('CNN_Classification_model_nlms.h5')
cnn_rls_model = models.load_model('CNN_Classification_model_rls.h5')

In [ ]:
label_names = ['AF', 'N']

In [ ]:
dataset_folder ="dataset/"
filenames = []
for filename in os.listdir(dataset_folder):
    if filename.find("AFDB_pasien_") > -1:
        filenames.append(filename)

In [ ]:
filenames

- load inference data

In [ ]:
inference_data = {}
for name in filenames:
    infer = pd.read_csv(dataset_folder + name, header=None)
    inference_data[name] = infer

In [ ]:
inference_data[filenames[0]].head()

In [ ]:
mat_infer_data = {}
for name in filenames:
    infer_df = inference_data[name]
    mat_infer_data[name] = infer_df.values

- plot histogram

In [ ]:
def plot_hist(df, name, min_, size, fs=250):
    # filter data berdasarkan class_number 
    ECG=df.values
    
    # scale tm value (*1000) agar bisa ditampilkan dalam satuam (ms)
    tm = np.arange(ECG.shape[1], dtype='float') / fs * 1000
    
    # ambil data hanya pada kolom `min_` sampai `size`
    # convert ke 1D array
    ECG=ECG[:,min_:size]
    ECG_flatten=ECG.flatten()
    
    # generate time data untuk tiap kolom sample, 
    # ulang sebanyak ECG.shape[0]
    times = np.tile(tm[min_:size], ECG.shape[0]) 

    # plot sebagai 2D histogram
    plt.hist2d(times, ECG_flatten, bins=(80,80), cmap=plt.cm.jet)
    plt.title("2D histogram on data - " + name)
    plt.xlabel("Time (ms)")
    plt.ylabel("Normalized Value")
    plt.show()

In [ ]:
for fn in filenames:
    df = inference_data[fn]
    plot_hist(df, fn, min_=0, size=80)

- Apply denoising model to inference data

In [ ]:
lms_den_res = {}
nlms_den_res = {}
rls_den_res = {}
for name in filenames:
    lms_den_res[name] = lms_model.transform(mat_infer_data[name])
    nlms_den_res[name] = nlms_model.transform(mat_infer_data[name])
    rls_den_res[name] = rls_model.transform(mat_infer_data[name])
    print("denoising data - %s" % name)

In [ ]:
def plot_denoising_test(y, x, model='lms', fs=250):
    times = np.arange(y.shape[0]) / fs * 1000 
    # show results
    plt.figure(figsize=(6,4))
    plt.title("Adaptation Result %s - Inference Dataset" % model.upper())
    plt.xlabel("time (ms)")
    plt.plot(times, y,"b", label="y - denoised signal")
    plt.plot(times, x,"g", label="x - original signal")
    plt.tight_layout()
    plt.legend()
    plt.grid()
    plt.show()

- Check result

In [ ]:
name = filenames[0]
idx = 246

y = lms_den_res[name][idx]
x = mat_infer_data[name][idx]

plot_denoising_test(y, x, model='lms')

## Reshape matrix result of denoising

In [ ]:
lms_mat_3d = {}
nlms_mat_3d = {}
rls_mat_3d = {}
for name in filenames:
    lms_mat_3d[name] = lms_den_res[name].reshape(len(lms_den_res[name]), lms_den_res[name].shape[1], 1)
    nlms_mat_3d[name] = nlms_den_res[name].reshape(len(nlms_den_res[name]), nlms_den_res[name].shape[1], 1)
    rls_mat_3d[name] = rls_den_res[name].reshape(len(rls_den_res[name]), rls_den_res[name].shape[1], 1)


- Apply to CNN model

In [ ]:
cnn_lms_classif_result = {}
cnn_nlms_classif_result = {}
cnn_rls_classif_result = {}
for name in filenames:
    cnn_lms_classif_result[name] = cnn_lms_model.predict(lms_mat_3d[name])
    cnn_nlms_classif_result[name] = cnn_nlms_model.predict(nlms_mat_3d[name])
    cnn_rls_classif_result[name] = cnn_rls_model.predict(rls_mat_3d[name])
    print("predicting data - %s" % name)

- Check result

In [ ]:
def plot_cnn_result(data, label='lms', confidence=0.0, fs = 250):
    times = np.arange(data.shape[0]) / fs * 1000 
    confidence = confidence*100
    # show results
    plt.figure(figsize=(6,4))
    plt.title("Classification Result : %s - confidence (%.2f%%)" % (label, confidence))
    plt.xlabel("time (ms)")
    plt.plot(times, data,"g", label="signal")
    plt.tight_layout()
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
pred_result = []
for name in filenames :
    cnn_res = cnn_rls_classif_result[name] if denoising == 'rls' else cnn_nlms_classif_result[name] if denoising == 'nlms' else cnn_lms_classif_result[name]
    print("\n\n\nPrediction Result - %s\n" % name)
    for i, res in enumerate(cnn_res):
        predicted_id = np.argmax(res)
        label = label_names[predicted_id]
        confidence = res[predicted_id]
        label = label if confidence > 0.40 else '?' # threshold for confidence level , set to 70%
        pred_result.append([name, i, label, confidence])
        
        print('%s \t index - %d \t %s \t %s \t %.2f %%' % (name, i, label, labels[i], confidence*100))

In [ ]:
len(cnn_res), len(labels)

In [ ]:
name = filenames[0]
idx = np.where(name == np.array(pred_result)[:, 0])[0]

In [ ]:
res = pred_result[idx[30]]
data = lms_den_res[name][res[1]]
plot_cnn_result(data, res[2], res[3])

In [ ]:
res = pred_result[idx[-3]]

data = lms_den_res[name][res[1]]
plot_cnn_result(data, res[2], res[3])

In [ ]:
pred_result_df = pd.DataFrame(pred_result, columns=['filename', 'idx', 'label', 'confidence'])

In [ ]:
pred_result_df.head()

In [ ]:
for name in filenames:
    print(name)
    count = dict(pred_result_df[pred_result_df['filename'] == name]['label'].value_counts())
    for key in count :
        print("%s \t: %d" % (key, count[key]))
    print('\n')

In [ ]:
def plot_donat(porsion, title, labels, colors=['red', 'green', 'blue']):
    plt.figure(figsize=(10,5))

    circle = plt.Circle( (0,0), 0.7, color='white')
    plt.pie(porsion, 
            labels=labels, 
            colors=colors,
            autopct='%1.1f%%')

    plt.title(title)
    p = plt.gcf()
    p.gca().add_artist(circle)
    plt.show()

In [ ]:
for name in filenames:
    count = dict(pred_result_df[pred_result_df['filename'] == name]['label'].value_counts())
    plot_donat(count.values(), name, count.keys())

- save result as csv

In [ ]:
pred_result_df.to_csv(dataset_folder + 'inference_result_cnn_%s_pasien_AF_dan_Normal.csv' % denoising)